In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
copy_number_gene_x_cell_line = pd.read_csv("CCLE_gene_cn.csv", index_col=0).T

copy_number_gene_x_cell_line.index = (
    str_.split()[0] for str_ in copy_number_gene_x_cell_line.index
)

assert not copy_number_gene_x_cell_line.index.has_duplicates

copy_number_gene_x_cell_line.index.name = "Gene"

copy_number_gene_x_cell_line.columns = kraft.normalize_cell_line_names(
    copy_number_gene_x_cell_line.columns
)

assert not copy_number_gene_x_cell_line.columns.has_duplicates

copy_number_gene_x_cell_line.to_csv("copy_number.gene_x_cell_line.tsv", sep="\t")

copy_number_gene_x_cell_line

In [ ]:
def split_copy_numbers(copy_numbers, threshold_loss, threshold_gain):

    vector = np.full(copy_numbers.size, np.nan)

    vector[copy_numbers.values < threshold_loss] = -1

    vector[threshold_gain < copy_numbers.values] = 1

    gene = copy_numbers.name

    dataframe = kraft.binarize(pd.Series(vector, index=copy_numbers.index, name=gene))

    dataframe = dataframe.loc[dataframe.index.dropna()]

    dataframe.index = dataframe.index.map(
        {-1: "{} -".format(gene), 1: "{} +".format(gene)}
    )

    return dataframe

In [ ]:
mean = np.nanmean(copy_number_gene_x_cell_line.values)

std = np.nanstd(copy_number_gene_x_cell_line.values)

threshold_loss = mean - 3 * std

threshold_gain = mean + 3 * std

threshold_loss, threshold_gain

In [ ]:
file_path = "gene_loss_gain_x_cell_line.tsv"

n = copy_number_gene_x_cell_line.shape[0]

n_per_print = n // 100

with open(file_path, mode="w") as io:

    for i, (gene, copy_numbers) in enumerate(copy_number_gene_x_cell_line.iterrows()):

        if i % n_per_print == 0:

            print("{}/{} ...".format(i + 1, n))

        split_copy_numbers(copy_numbers, threshold_loss, threshold_gain).to_csv(
            io, sep="\t", header=i == 0, mode="a"
        )

gene_gain_loss_x_cell_line = pd.read_csv(file_path, sep="\t", index_col=0)

gene_gain_loss_x_cell_line

In [ ]:
gene_gain_loss_x_cell_line.sum(axis=1).sort_values(ascending=False)